### Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cobra
from cobra.io import read_sbml_model,save_json_model

### Load xml model + screen exchanges

In [2]:
mini_fbc2_path = "iCN1361.xml" ## Change filename to run a different model
model = read_sbml_model(mini_fbc2_path)

In [3]:
## Get all metabolites and reactions in the model to check for desired substrates
metabolite_ids = [i.id for i in model.metabolites]
reaction_ids = [r.id for r in model.reactions]

In [4]:
### Get all exchange reactions in the model

exchange_reactions = []
for rxn in reaction_ids:
    if('EX_' in rxn):
        exchange_reactions.append(rxn)

In [10]:
### List of all substrates and corresponding reactions to screen for growth
screening_substrates = ['M_FORMATE','M_D_LACTATE','M_L_LACTATE','M_AMMONIUM']
screening_uptakes = ['EX_FORMATE_e','EX_D_LACTATE_e','EX_L_LACTATE_e','EX_AMMONIUM_e']


with model:
    
    # prevent fructose uptake
    model.reactions.get_by_id('EX_BETA_D_FRUCTOSE_e').upper_bound=0
    model.reactions.get_by_id('EX_BETA_D_FRUCTOSE_e').lower_bound=0
    
    for uptake in screening_uptakes:
        model.reactions.get_by_id(uptake).lower_bound = -100
        model.reactions.get_by_id(uptake).upper_bound = 1
        
    #prevent any D_Lactate consumption
    model.reactions.get_by_id('EX_D_LACTATE_e').upper_bound=0
    model.reactions.get_by_id('EX_D_LACTATE_e').lower_bound=0
    
    ##limit ammonia
    '''
    Uptake = 0 returns 0 biomass since there is no other 'N' source, so add in uptake of other exchanges
    '''
    model.reactions.get_by_id('EX_AMMONIUM_e').lower_bound=0
    
    # Checking if any of the amino acid uptakes results in N
    for rxn in exchange_reactions:
        model.reactions.get_by_id(rxn).lower_bound = -20
    
    #Solve for biomass
    soln = model.optimize()
    
soln

,fluxes,reduced_costs
R_RXN_12508,0.000000,0.000000e+00
EX_TYR_e,-20.000000,-2.388146e-02
R_RXN_8899,0.000000,-4.049995e-18
R_SHIKIMATE_5_DEHYDROGENASE_RXN,0.000000,2.388146e-01
R_NAG6PDEACET_RXN,20.000000,-3.577906e-19
...,...,...
EX_NO_e,0.000000,0.000000e+00
R_OCTAPRENYL_METHYL_METHOXY_BENZOQ_OH_RXN_AN,0.057347,8.658505e-17
R_RXN_21483,0.067857,2.775558e-16
R_GLYCOLATEDEHYDRO_RXN,0.094036,-2.220446e-16


In [12]:
### Check which exchanges are being uptaken for growth
optimal_exchange_fluxes = soln.fluxes.loc[exchange_reactions]
optimal_exchange_fluxes.loc[optimal_exchange_fluxes>0]

EX_FORMATE_e                1.000000
EX_PHE_e                   54.864431
EX_VAL_e                   20.343327
EX_SUC_e                  169.223517
EX_PROTON_e               124.287106
EX_ACET_e                 372.643940
EX_CARBON_DIOXIDE_e       339.233463
EX_HYDROGEN_MOLECULE_e     37.824914
EX_Pi_e                    27.639046
EX_CARBON_MONOXIDE_e        0.098740
EX_UREA_e                  66.708275
EX_HYPOXANTHINE_e          14.898878
EX_CH33ADO_e                0.098740
EX_NITROGEN_MOLECULE_e     20.000000
EX_ILE_e                   51.820431
Name: fluxes, dtype: float64

In [7]:
#pfba_solution = cobra.flux_analysis.pfba(model)

In [15]:
model.reactions.get_by_id('EX_HYPOXANTHINE_e')

Reaction identifier,EX_HYPOXANTHINE_e
Name,HYPOXANTHINE transporter
Memory address,0x29872cdc0
Stoichiometry,M_HYPOXANTHINE --> hypoxanthine -->
GPR,
Lower bound,0.0
Upper bound,1000.0
